In [7]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

test_data = pd.read_csv('../../../../mnt/sda/suhohan/emnist/emnist-byclass-test.csv')
num_test_samples = test_data.shape[0]

x_test = test_data.iloc[:, 1:].to_numpy().reshape((num_test_samples, 28, 28, 1))
y_test = tf.keras.utils.to_categorical(test_data.iloc[:, 0], 62)
y_test_int = test_data.iloc[:, 0].to_numpy()

x_test_corrected = np.array([np.fliplr(np.rot90(img.reshape(28, 28), k=3))
                            for img in x_test]).reshape(num_test_samples, 28, 28, 1)

In [ ]:
x_test_resized = tf.image.resize(x_test_corrected, [32, 32])
x_test_resized = tf.cast(x_test_resized, tf.float32)
x_test_resized_np = x_test_resized.numpy()

In [8]:
def get_latest_model_file(checkpoint_dir):
    files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.hdf5')]
    files.sort(key=lambda x: os.path.getmtime(os.path.join(checkpoint_dir, x)), reverse=True)
    return os.path.join(checkpoint_dir, files[0]) if files else None

In [9]:
def label_to_char(label):
    if 0 <= label <= 9:
        return str(label)
    elif 10 <= label <= 35:
        return chr(label + 87)
    elif 36 <= label <= 61:
        return chr(label + 29)
    else:
        raise ValueError(f"Unexpected label: {label}")

# LeNet-5

In [ ]:
checkpoint_dir = './checkpoints/checkpoints_lenet/'
latest_model_file = get_latest_model_file(checkpoint_dir)

if latest_model_file:
    print(f"Loading model from: {latest_model_file}")
    model = tf.keras.models.load_model(latest_model_file)
    loss_lenet, acc_lenet = model.evaluate(x_test_corrected, y_test_int)
    print("Loss:", loss_lenet)
    print("Accuracy:", acc_lenet)
else:
    print("No model files found in the checkpoint directory.")

In [ ]:
num_samples = 200
sample_indices = np.random.choice(num_test_samples, num_samples, replace=False)
sample_images = x_test_corrected[sample_indices]
sample_labels = y_test_int[sample_indices]

predicted_labels = model.predict(sample_images)
predicted_labels_int = np.argmax(predicted_labels, axis=1)

correct_indices = np.where(predicted_labels_int == sample_labels)[0]
incorrect_indices = np.where(predicted_labels_int != sample_labels)[0]

correct_images = sample_images[correct_indices]
correct_labels = sample_labels[correct_indices]
correct_predictions = predicted_labels_int[correct_indices]

incorrect_images = sample_images[incorrect_indices]
incorrect_labels = sample_labels[incorrect_indices]
incorrect_predictions = predicted_labels_int[incorrect_indices]

num_display = 10
plt.figure(figsize=(20, 10))

for i in range(min(num_display, len(correct_images))):
    plt.subplot(2, num_display, i + 1)
    plt.imshow(correct_images[i].reshape(28, 28), cmap='gray')
    true_label = label_to_char(correct_labels[i])
    predicted_label = label_to_char(correct_predictions[i])
    plt.title(f"Answer: {true_label}\nPredicted: {predicted_label}", fontsize=10)
    plt.axis('off')

for i in range(min(num_display, len(incorrect_images))):
    plt.subplot(2, num_display, num_display + i + 1)
    plt.imshow(incorrect_images[i].reshape(28, 28), cmap='gray')
    true_label = label_to_char(incorrect_labels[i])
    predicted_label = label_to_char(incorrect_predictions[i])
    plt.title(f"Answer: {true_label}\nPredicted: {predicted_label}", fontsize=10)
    plt.axis('off')

plt.tight_layout()
plt.show()

# ResNet50

In [ ]:
checkpoint_dir = './checkpoints/checkpoints_resnet/'

latest_model_file = get_latest_model_file(checkpoint_dir)
if latest_model_file:
    print(f"Loading model from: {latest_model_file}")
    model = tf.keras.models.load_model(latest_model_file)
    loss_resnet, acc_resnet = model.evaluate(x_test_resized_np, y_test_int)
    print("Loss:", loss_resnet)
    print("Accuracy:", acc_resnet)
else:
    print("No model files found in the checkpoint directory.")

In [ ]:
num_samples = 200
sample_indices = np.random.choice(num_test_samples, num_samples, replace=False)
sample_images = x_test_resized_np[sample_indices]
sample_labels = y_test_int[sample_indices]

predicted_labels = model.predict(sample_images)
predicted_labels_int = np.argmax(predicted_labels, axis=1)

correct_indices = np.where(predicted_labels_int == sample_labels)[0]
incorrect_indices = np.where(predicted_labels_int != sample_labels)[0]

correct_images = sample_images[correct_indices]
correct_labels = sample_labels[correct_indices]
correct_predictions = predicted_labels_int[correct_indices]

incorrect_images = sample_images[incorrect_indices]
incorrect_labels = sample_labels[incorrect_indices]
incorrect_predictions = predicted_labels_int[incorrect_indices]

num_display = 10
plt.figure(figsize=(20, 10))

for i in range(min(num_display, len(correct_images))):
    plt.subplot(2, num_display, i + 1)
    plt.imshow(correct_images[i].reshape(32, 32), cmap='gray')
    true_label = label_to_char(correct_labels[i])
    predicted_label = label_to_char(correct_predictions[i])
    plt.title(f"Answer: {true_label}\nPredicted: {predicted_label}", fontsize=10)
    plt.axis('off')

for i in range(min(num_display, len(incorrect_images))):
    plt.subplot(2, num_display, num_display + i + 1)
    plt.imshow(incorrect_images[i].reshape(32, 32), cmap='gray')
    true_label = label_to_char(incorrect_labels[i])
    predicted_label = label_to_char(incorrect_predictions[i])
    plt.title(f"Answer: {true_label}\nPredicted: {predicted_label}", fontsize=10)
    plt.axis('off')

plt.tight_layout()
plt.show()

# Our Model

In [14]:
checkpoint_dir = '../checkpoints/checkpoints_final'

latest_model_file = get_latest_model_file(checkpoint_dir)
if latest_model_file:
    print(f"Loading model from: {latest_model_file}")
    model = tf.keras.models.load_model('../checkpoints/checkpoints_ours_2/weights.1716893020.hdf5')
    loss_our, acc_our = model.evaluate(x_test, y_test_int)
    print("Loss:", loss_our)
    print("Accuracy:", acc_our)
else:
    print("No model files found in the checkpoint directory.")

In [ ]:
num_samples = 30
sample_indices = np.random.choice(num_test_samples, num_samples, replace=False)
sample_images = x_test[sample_indices]
sample_labels = y_test_int[sample_indices]
print(sample_images.shape)
predicted_labels = model.predict(sample_images)
predicted_labels_int = np.argmax(predicted_labels, axis=1)

correct_indices = np.where(predicted_labels_int == sample_labels)[0]
incorrect_indices = np.where(predicted_labels_int != sample_labels)[0]

correct_images = sample_images[correct_indices]
correct_labels = sample_labels[correct_indices]
correct_predictions = predicted_labels_int[correct_indices]

incorrect_images = sample_images[incorrect_indices]
incorrect_labels = sample_labels[incorrect_indices]
incorrect_predictions = predicted_labels_int[incorrect_indices]

num_display = 10
plt.figure(figsize=(20, 10))

for i in range(min(num_display, len(correct_images))):
    plt.subplot(2, num_display, i + 1)
    plt.imshow(np.flip(np.rot90(correct_images[i].reshape(28, 28), 3)), cmap='gray')
    true_label = label_to_char(correct_labels[i])
    predicted_label = label_to_char(correct_predictions[i])
    plt.title(f"Answer: {true_label}\nPredicted: {predicted_label}", fontsize=10)
    plt.axis('off')

for i in range(min(num_display, len(incorrect_images))):
    plt.subplot(2, num_display, num_display + i + 1)
    plt.imshow(incorrect_images[i].reshape(28, 28), cmap='gray')
    true_label = label_to_char(incorrect_labels[i])
    predicted_label = label_to_char(incorrect_predictions[i])
    plt.title(f"Answer: {true_label}\nPredicted: {predicted_label}", fontsize=10)
    plt.axis('off')

plt.tight_layout()
plt.show()